## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [22]:
import pandas as pd
credit_data = pd.read_csv('/datasets/data.csv')
print(credit_data.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Для начала анализа данных в таблице опишем первое впечатление по каждому столбцу:
1. children - количество детей в семье.
Тип данных int. На первый взгляд вопросов не возникает.
2. days_employed - общий трудовой стаж в днях.
Тип данных float. Есть две проблемы: отрицательные значения и очень большие значения (более 100лет). Их необходимо заменить. Также есть пропуски.
3. dob_years - возраст клиента в годах
Тип данных int. На первый взгляд вопросов не возникает.
4. education — уровень образования клиента.
Тип данных object. Необходимо проверить столбец на уникальность, т.к. есть слова с "большой буквы".
5. education_id — идентификатор уровня образования
Тип данных int. На первый взгляд вопросов не возникает.
6. family_status — семейное положение
Тип данных object. На первый взгляд вопросов не возникает.
7. family_status_id — идентификатор семейного положения
Тип данных int. На первый взгляд вопросов не возникает.
8. gender — пол клиента
Тип данных object. На первый взгляд вопросов не возникает.
9. income_type — тип дохода
Тип данных object. На первый взгляд вопросов не возникает.
10. debt — имел ли задолженность по возврату кредитов
Тип данных int. На первый взгляд вопросов не возникает.
11. total_income — ежемесячный доход
Тип данных float. Основная проблема - есть пропуски.
12. purpose - цель получения кредита.
Тип данных object. Много разных целей с одним смыслом.

Далее будет проведен более глубокий анализ.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [23]:
print('Количество пропусков в таблице:\n' , credit_data.isnull().sum())

# Отсортируем таблицу по столбцу days_employed по возрастанию, выведем часть столбцов:
print('Сортировка по возрастанию стажа:')
print(credit_data[['children', 'days_employed' , 'dob_years' , 'total_income' ,'income_type']].sort_values(by = 'days_employed' , ascending = True))
print()

# Определяем стаж с отрицательным значением
print('Количество отрицательных значений стажа до замены:' , 
      credit_data['days_employed'][credit_data['days_employed'] < 0].count())
#Изменим знак всего столбца на положительный.
credit_data['days_employed'] = abs(credit_data['days_employed'])
print('Количество отрицательных значений стажа после замены:' , 
credit_data['days_employed'][credit_data['days_employed'] < 0].count())
# Определяем нулевой возраст
print('Сортировка по возрастанию возраста:')
print(credit_data[['children', 'days_employed' , 'dob_years' , 'total_income' ,'income_type']].
      sort_values(by = 'dob_years' , ascending = True))
print()
print('Количество нулевых значений возраста:' , 
      credit_data['dob_years'][credit_data['dob_years'] == 0].count())

# Заменим нулевые значения возраста средним значением возраста по типу дохода, 
#т.к. они нам пригодятся для замены пропусков стажа работы
median_years = credit_data.groupby('income_type')['dob_years'].median()  
print()
print()
income_type_array = ['безработный' , 'в декрете' , 'госслужащий' ,
                     'компаньон' , 'пенсионер' , 'предприниматель' , 'сотрудник' , 'студент']
for i in range(len(income_type_array)):
    credit_data.loc[((credit_data['dob_years'] == 0) &
                     (credit_data['income_type'] == income_type_array[i])), 'dob_years'] = median_years[i]    
print('Отсортируем таблицу по возрасту и проверим исчезли ли нулевые значения:')
print(credit_data[['children', 'days_employed' , 'dob_years' , 'total_income' , 'income_type']].sort_values(by = 'dob_years' , ascending = True))

# Замена пропусков
# Для удобства выделим из таблицы только интересующие столбцы - возраст, стаж , должность 
credit_data_full = credit_data[['days_employed' , 'dob_years' , 'income_type' , 'total_income']]
print(credit_data_full)

#Отсекаем пропуски и нулевые значения возраста
credit_data_full = credit_data_full[(credit_data_full['days_employed'] > 0)]
#Проверим попали ли пропуски 
print('Проверим есть ли пропуски в новой таблице:')
print(credit_data_full.isnull().sum())
# Напишем функцию для определения стажа по возрастной группе. Стаж определяем медианой
def median_days_employed(age_min , age_max):     
    median_employed = credit_data_full['days_employed'][(credit_data_full['dob_years'] >= age_min) & (credit_data_full['dob_years'] < age_max)].median()
    return median_employed


#Заменяем пропуски стажа медианой стажа для группы возраста
credit_data.loc[(credit_data['days_employed'].isna() 
                 & (credit_data['dob_years'] >= 19) & (credit_data['dob_years'] < 30)) , 'days_employed'] = median_days_employed(19 , 30)
credit_data.loc[(credit_data['days_employed'].isna() 
                 & (credit_data['dob_years'] >= 30) & (credit_data['dob_years'] < 40)) , 'days_employed'] = median_days_employed(30 , 40)
credit_data.loc[(credit_data['days_employed'].isna() 
                 & (credit_data['dob_years'] >= 40) & (credit_data['dob_years'] < 50)) , 'days_employed'] = median_days_employed(49 , 50)
credit_data.loc[(credit_data['days_employed'].isna() 
                 & (credit_data['dob_years'] >= 50) & (credit_data['dob_years'] < 60)) , 'days_employed'] = median_days_employed(59 , 60)
credit_data.loc[(credit_data['days_employed'].isna() 
                 & (credit_data['dob_years'] >= 60) & (credit_data['dob_years'] < 80)) , 'days_employed'] = median_days_employed(69 , 80)
print()
print('Проверим наличе пропусков после замены:')
print( credit_data.isna().sum())

# Заменим пропуски дохода медианным значением по типу дохода.
# Исключаем из расчета среднего строки с пропусками
credit_data_income = credit_data.dropna()
median_income = credit_data_income.groupby('income_type')['total_income'].median()  
print('Медианный доход по типу занятости:')
print(median_income)
print()

income_type_array = ['безработный' , 'в декрете' , 'госслужащий' ,
                     'компаньон' , 'пенсионер' , 'предприниматель' , 'сотрудник' , 'студент']
for i in range(len(income_type_array)):
    credit_data.loc[((credit_data['total_income'].isna())
                     & (credit_data['income_type'] == income_type_array[i])), 'total_income'] = median_income[i]    

print('Проверим наличие пропусков после замены:')
print( credit_data.isna().sum())


 
#Изменим знак всего столбца children на положительный.
print('Отрицательные значение в столбце children сделаем положительными: ')

credit_data['children'] = abs(credit_data['children'])
credit_data

Количество пропусков в таблице:
 children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Сортировка по возрастанию стажа:
       children  days_employed  dob_years   total_income income_type
16335         1  -18388.949901         61  186178.934089   сотрудник
4299          0  -17615.563266         61  122560.741753   компаньон
7329          0  -16593.472817         60  124697.846781   сотрудник
17838         0  -16264.699501         59   51238.967133   сотрудник
16825         0  -16119.687737         64   91527.685995   сотрудник
...         ...            ...        ...            ...         ...
21489         2            NaN         47            NaN   компаньон
21495         1            NaN         50            NaN   сотрудник


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод


Явные пропуски определены в столбцах 'days_employed' и 'total_income'. Неявные пропуски обнаружены в столбце 'dob_years' в виде возраста с нулевым значением. Также обнаружены ошибки в столбце 'days_employed' в виде отрицательных и аномальных значений стажа работы. Возможная причины появления пропусков - человеческий фактор, данные были не заполнены. 
Также обнаружены отрицательные значения в столбце children.

Решения: 
  - все отрицательные значения 'days_employed' делаем положительными командой abs(), т.к. минус мог быть ошибкой
 
  
  
  - нулевые значения столбца "dob_years" заменяем медианой значений возраста в зависимости от типа занятости.
  - предположив, что стаж в основном зависит от возраста, заменим пропущенные значения в столбце 'days_employed' медианой стажа для группы возраста. Приняты следующие группы: 19-30, 30-40, 40-50, 50-60 , 60-80. Для каждой группы определяем медиану стажа. Медиана принята в связи с большим разбросом значений стажа.
  - 'total_income' - группировкой определяем медиану по каждому типу занятости, и заменяем пропущенные значения в 'total_income'. Медиана принята в связи с большим разбросом значений.
         
  - отрицательные значения children исправляем на положительные abs().

### Замена типа данных

In [24]:
#Для исключения вещественных значений заменим тип float64 на int64 столбцов days_employed и total_income
credit_data['days_employed'] = credit_data['days_employed']. astype('int')
credit_data['total_income']= credit_data['total_income']. astype('int')
credit_data['dob_years']= credit_data['dob_years']. astype('int')
#Проверим результат замены
print(credit_data.info())
credit_data[['days_employed' , 'total_income' , 'dob_years' ]].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


,days_employed,total_income,dob_years
0,8437,253875,42
1,4024,112080,36
2,5623,145885,33
3,4124,267628,32
4,340266,158616,53


### Вывод


Для удобства работы с таблицей и сокращения использования памяти заменим вещественные числа на целые методом astype(). 
С учетом преобразований на предыдущем этапе всего три столбца с вещественными значениями - days_employed , dob_years,  total_income. 
Результат преобразований выведен на экран.


### Обработка дубликатов

In [25]:
#Определим дубликаты:
print('Количество дубликатов в таблице:')

print(credit_data.duplicated().sum())
credit_data = credit_data.drop_duplicates().reset_index(drop = True)
print('Проверим количество дубликатов после удаления:')
print(credit_data.duplicated().sum())

#Определим количество уникальных значений в столбце education:

print('Уникальные значения в столбце education:')
print(credit_data['education'].value_counts())

print()
print('Количество уникальных значений в столбце family_status:')
print(credit_data['family_status'].value_counts())
print()
#Приведем все значения family_status и education к нижнему регистру
credit_data['family_status'] = credit_data['family_status'].str.lower()
credit_data['education'] = credit_data['education'].str.lower()
print('Количество уникальных значений в столбце education после преобразования:')
print(credit_data['education'].value_counts())


print('Количество уникальных значений в столбце purpose:')
print(credit_data['purpose'].value_counts())
print()

print('Количество уникальных значений в столбце income_type:')
print(credit_data['income_type'].value_counts())
print()

print('Количество уникальных значений в столбце gender:')
print(credit_data['gender'].value_counts())
credit_data = credit_data[credit_data['gender'] != 'XNA']
print()

# Группируем столбцы education  и education_id
print(credit_data.groupby(['education' , 'education_id'])['education_id'].count())
print()

# Группируем столбцы family_status и family_status_id
print(credit_data.groupby(['family_status' , 'family_status_id'])['family_status_id'].count())
print()

print('Количество дубликатов в таблице после приведения к нижнему регистру:')

print(credit_data.duplicated().sum())
credit_data = credit_data.drop_duplicates().reset_index(drop = True)
print('Проверим количество дубликатов после удаления:')
print(credit_data.duplicated().sum())
print('Информация по таблице после преобразований')
print(credit_data.info())

Количество дубликатов в таблице:
54
Проверим количество дубликатов после удаления:
0
Уникальные значения в столбце education:
среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Количество уникальных значений в столбце family_status:
женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Количество уникальных значений в столбце education после преобразования:
среднее                15188
высшее                  5251
неоконченное высшее      744
на

### Вывод


1. Методом duplicated() обнаружены 54 дубликата  в исходной таблице и 17 после приведения к нижнему регистру.Методом drop_duplicates() дубликаты удалены.
2. Анализом уникальных значений столбца education обнаружены дубликаты данных с разным регистром букв. Возможная причина - человеческий фактор при занесении данных из анкет в электронный вид.
Методом str.lower() все значения приведены к нижнему регистру. В итоге получены 5 значений столбца  education.
3. В столбце family_status всего 5 значений и все уникальные. Для порядка также их преобразуем к нижнему регистру, для исключения возможных дублей
4. В столбцы 'gender' обнаружено одно значение XNA. Т.к. одно значение никак не повлияет на общий результат, то всю строку со значение XNA  можно удалить.
5. В столбце income_type есть значения с количеством людей 2 или 1. Их удалять не будем. Они не мешают и на данные не влияют. 
6. При группировке столбцов family_status / family_status_id  и education / education_id ошибок не обнаружено.
7. Проверка на уникальность значений столбца с целями purpose показала множество одинаковых по смыслу слов с разной формулировкой. Для этого столбца необходимо провести лемматизацию.

### Лемматизация

In [26]:
# создаем новый столбец с леммами
from pymystem3 import Mystem
m = Mystem()
credit_data['lemmas'] = credit_data['purpose'].apply(m.lemmatize)

#Функция проверки слова и записи в новый столбец
def text_func1(row):
    lemma_text = row['lemmas']
    for word in lemma_text:
        if 'свадьба' == word :
            return 'свадьба'    
        if 'автомобиль' == word:
            return 'автокредит'
        if 'образование' == word:
            return 'образование' 
        if 'жилье'== word or 'недвижимость' == word:
            return 'ипотека'          


### Вывод

    
    

С помощью леммматизации создан столбец с массивом леммы слов целей кредита. 

### Категоризация данных

In [27]:
credit_data['categorized_purpose'] = credit_data.apply(text_func1 , axis=1)    
print(credit_data['categorized_purpose'].head(5))
print()
print('Количество уникальных значений в столбце categorized_purpose после преобразования:')
print()
print(credit_data['categorized_purpose'].value_counts())
credit_data.head(10)

#Создадим словари по вопросам задания
#Зависимость задолженностей от детей
print('Словарь по количеству детей:')
dict_children = credit_data['children']
#Удалим дубликаты
dict_children = dict_children.drop_duplicates().sort_values(ascending = True).reset_index(drop = True)
print(dict_children)
print('Словарь по семейному статусу:')
dict_family = credit_data['family_status']
#Удалим дубликаты
dict_family = dict_family.drop_duplicates().sort_values(ascending = True).reset_index(drop = True)
print(dict_family)
print('Словарь по типу занятости (для анализа дохода):')

dict_income = credit_data['income_type']
#Удалим дубликаты
dict_income = dict_income.drop_duplicates().sort_values( ascending = True).reset_index(drop = True)
print(dict_income)
print('Словарь по целям кредита:')
dict_purpose = credit_data['categorized_purpose']

dict_purpose = dict_purpose.drop_duplicates().sort_values(ascending = True).reset_index(drop = True)
print(dict_purpose)


0        ипотека
1     автокредит
2        ипотека
3    образование
4        свадьба
Name: categorized_purpose, dtype: object

Количество уникальных значений в столбце categorized_purpose после преобразования:

ипотека        10810
автокредит      4306
образование     4013
свадьба         2324
Name: categorized_purpose, dtype: int64
Словарь по количеству детей:
0     0
1     1
2     2
3     3
4     4
5     5
6    20
Name: children, dtype: int64
Словарь по семейному статусу:
0                в разводе
1           вдовец / вдова
2         гражданский брак
3          женат / замужем
4    не женат / не замужем
Name: family_status, dtype: object
Словарь по типу занятости (для анализа дохода):
0        безработный
1          в декрете
2        госслужащий
3          компаньон
4          пенсионер
5    предприниматель
6          сотрудник
7            студент
Name: income_type, dtype: object
Словарь по целям кредита:
0     автокредит
1        ипотека
2    образование
3        свадьба
Name: ca

### Вывод
 

Функцией сделана группировка целей по лемме.
Таким образом выделены 4 цели кредита: 
- ипотека
- автокредит
- свадьба
- образование

Новые цели записываются в столбец 'categorized_purpose'.

Созданы "словари" по количеству детей, семейному статусу, типу занятости и цели кредита.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [28]:
pivot_children = credit_data.pivot_table(index = ['children'  ] , columns = 'debt' , values = 'family_status' , aggfunc = 'count')
#Сохраним в отдельную таблицу толдько интересующие нас столбцы

pivot_children['ratio_debt_1'] = round(pivot_children[1] / pivot_children[0] , 2)
pivot_children


debt,0,1,ratio_debt_1
children,,,
0,13027.0,1063.0,0.08
1,4410.0,445.0,0.10
2,1858.0,194.0,0.10
3,303.0,27.0,0.09
4,37.0,4.0,0.11
5,9.0,NaN,NaN
20,68.0,8.0,0.12


### Вывод
Для оценки влияния детей в сводную таблицу вводим столбец относительного количества задолженностей
Различие между просрочкой и возвратом вовремя  в среднем примерно 1%. Наличие детей и их количество практически не влияет на наличие задолженностей по кредиту.
А вот отсутствие детей показывает показывает большее количество возвратов в срок.

Наличие детей и их количество практически не влияет на наличие задолженностей по кредиту.
А вот отсутствие детей показывает показывает большее количество возвратов в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
pivot_family = credit_data.pivot_table(index = ['family_status' ] , 
                                       columns = 'debt' , values = 'family_status_id' , aggfunc = 'count')

pivot_family['ratio_debt_1'] = round(pivot_family[1] / pivot_family[0] , 2)
pivot_family

debt,0,1,ratio_debt_1
family_status,,,
в разводе,1110,85,0.08
вдовец / вдова,896,63,0.07
гражданский брак,3762,388,0.10
женат / замужем,11408,931,0.08
не женат / не замужем,2536,274,0.11


### Вывод


Значения "не женат/не замужем" и "гражданский брак" очень близки чаще остальных групп имеют задолженности по выплате кредита. В целом как семейные клиенты, так и разведенные одинаково выплачивают кредит.
Учитывая незначительные различия можно сделать вывод, что семейный статус мало влияет на возврат кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
pivot_income = round(credit_data.pivot_table(index = ['income_type' ] , columns = 'debt' , values = 'total_income' , aggfunc = 'mean') , 2)

pivot_income['ratio'] = round(pivot_income[0]/pivot_income[1], 2)
pivot_income

debt,0,1,ratio
income_type,,,
безработный,202722.00,59956.00,3.38
в декрете,NaN,53829.00,NaN
госслужащий,169919.26,152016.55,1.12
компаньон,199898.39,193848.57,1.03
пенсионер,135182.78,136385.07,0.99
предприниматель,499163.00,NaN,NaN
сотрудник,159990.94,155554.49,1.03
студент,98201.00,NaN,NaN


### Вывод


Для оценки принят средний доход для каждого типа занятости. 
По группам с большим количеством плательщиков различие минимальное. Максимальное различие в доходе 12% видно у госслужащих. Здесь видно, что больший доход способствует оплате кредита в срок. 
Остальные группы представлены 2 или 1 человеком, поэтому большое различие не показательно.


- Как разные цели кредита влияют на его возврат в срок?

In [31]:
pivot_purpose = credit_data.pivot_table(index = ['categorized_purpose' ] , 
                                       columns = 'debt' , values = 'family_status_id' , aggfunc = 'count')

pivot_purpose['ratio_debt_1'] = round(pivot_purpose[1] / pivot_purpose[0] , 2)
pivot_purpose

debt,0,1,ratio_debt_1
categorized_purpose,,,
автокредит,3903,403,0.10
ипотека,10028,782,0.08
образование,3643,370,0.10
свадьба,2138,186,0.09


### Вывод

Для автокредита и кредита на образование больше задолженнстей, чем для других целей. Для всех операций с жильем характерно меньшее количетво задолженностей.
Учитывая не значительные различия, не более 2%, можно сделать вывод, что цель кредита не влияет на его выплату в срок.


<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Все выводы правильные и у тебя отлично получается строить сводные таблицы, молодец
</div>



### Шаг 4. Общий вывод
  Обработка данных.
  1. Пропуски. Обнаружено 2174 пропуска данных (около 10%) в стаже и доходе, которые могли бы повлиять на принятие решений. Также обнаружены 101 нулевых значений в возрасте (0,4%), которые  не повлияли бы на результаты по причине их небольшого количества, но были обработаны, т.к. по возрасту определялась медиана стажа для заполнения пропусков в стаже.
  Для заполнения пропусков предположено, что стаж определяется возрастом, а доход типом занятости. По каждой группе возраста были определены медианы стажа и этим значением заполнены пропуски в соответствии с возрастом. По каждому типу занятости были определены медианы дохода и этим значением заполнены пропуски дохода в соответствии с группой занятости. Нулевые значения возраста заполнялись медианой возраста по типу занятости из предположения, что тип занятости в некоторых пределах определяется зависит от возраста.
  Для поиска пропусков применялся метод isna(). Заполнение пропусков - присвоением значений по методу доступа в ячейку loc(). В итоге: все пропуски и нулевые значения заполнены.
  2. Дубликаты. После приведения строковых данных столбцов с образованием и семейным статусом к нижнему регистру было обнаружено 71 дубликатов.
   Дубликаты удалены методом drop_duplicates().
  3. В столбце цели кредита обнаружено множество формулировок целей кредита. Для удобства обработки и анализа данных все цели сгруппированы в 4 значения: ипотека, автокредит, образование, свадьба. 
  Анализ данных:
  1. Зависимость возврата кредита в срок от количества детей.  
  Отсутствие детей на 2% положительнее сказывается на возврат кредит в срок по сравнению с теми, у кого есть дети. При этом количество детей не влияет на возврат кредита в срок. Незначительные различия в результатах говорят о том, что наличие детей является не показательным для определения вероятности возврата кредита в срок. Можно рассматреть этот показатель в комплексе с доходом и семейным статусом из предположения, что общий доход делится на большее количество членов семьи.
  
  2. Зависимость возврата кредита в срок от семейного положения.  
  Различия в возврате кредита между людьми с разным семейным положением ломает предположение о доходе на одного члена семьи, т.к. не женат/ не замужем и вдова/вдовец отличаются на 4%. Также необходимо рассматривать этот показатель в комплексе с доходом и другими обязательствами, например алименты в случае неженатого человека.  
 
  3. Зависимость возврата кредита в срок от уровня дохода.  
  Статистически определяются только 4 позиции: компаньон, сотрудник , госслужащий и пенсионер. Значительное различие между возвратившими и не возвратившими кредит в срок для госслужащих составляет 12%, что кажется определяет положительную зависимость. Но 3 других позиции не подтверждают этой тенденции - различие 4% совсем незначительно.
  
  4. Зависимость возврата кредита в срок от цели кредита.
  Максимальное различие в 2% между целями кредита однозначно показывает отсутствие зависисмости между возвратом кредита и целями кредита.

Главный вывод: как видно из анализа среднее количество не возвративших кредит составляет примерно 10% и оно не зависит от целей кредита, семейного статуса, наличия детей и , главное - дохода. Можно предположить, что в эту цифру попадают люди, например,  лишившиеся работы, или разбили машину в аварии и еще должны пострадавшему , и это не позволило заплатить за кредит. 
Правильнее было бы добавить столбец с кратким объяснением причины задолженности по предыдущим кредитам, участие в ДТП для автокредита, т.е. кредитную историю. 
  

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.